In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from spacy.tokenizer import Tokenizer
from spacy.lang.pt import Portuguese

In [ ]:
def replace_tokens(text, tokenised_text, keywords, punctuation, mode):
    keyword_replacements = []
    for i, t in enumerate(keywords):
        target_word = str(t)
        # ignore if it's just punctuation
        if target_word in punctuation:
            continue
        # ignore if it's not the list of BERT tokens
        if target_word.lower() not in tokenised_text:
            continue
            
        target_token = target_word.lower()
        masked_index = tokenised_text.index(target_token)
        tokenised_text[masked_index] = '[MASK]'

        indexed_tokens = tokeniser.convert_tokens_to_ids(tokenised_text)
        segments_ids = [1] * len(tokenised_text)
        segments_ids[0] = 0
        segments_ids[1] = 0

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        # Predict all tokens
        predictions = model(tokens_tensor, segments_tensors)

        chosen_token = "X"
        while True:
            predicted_index = torch.argmax(predictions[0, masked_index]).item()
            predicted_token = tokeniser.convert_ids_to_tokens([predicted_index])[0]
            if predicted_token not in punctuation and predicted_token.find(target_token) < 0 and target_token.find(predicted_token) < 0:
                try:
                    ri = keyword_replacements.index(predicted_token)
                except ValueError:
                    ri = -1
                if predicted_token == 'bell':
                    print("pt: " + predicted_token)
                    print(f'ri: {ri}')
                    print(f'kr: {keyword_replacements[ri]}')
                    print(f'k: {keywords[ri]}')
                    print(f'k: {keywords[i]}')
                if ri < 0 or (ri >= 0 and keywords[i] == keywords[ri]):
                    # and keyword_replacements[ri].find(predicted_token) < 0 and predicted_token.find(keyword_replacements[ri]) < 0)
                    chosen_word = predicted_token
                    print(f'{target_word} -> {chosen_word}')
                    break
                else:
                    predictions[0,masked_index,predicted_index] = -9999
            else:
                predictions[0,masked_index,predicted_index] = -9999

        if mode == 'all':
            current_text_tokens = spacy_tokeniser(text)
            text = ' '.join([str(spacy_tokeniser(chosen_word)) if str(token) == target_word else str(token) for token in current_text_tokens])
        else:
            text = text.replace(target_word, chosen_word, 1)
            
        tokenised_text = tokeniser.tokenize(text)
        keyword_replacements.append(chosen_word)
        
    return text

In [ ]:
model_path = "D:/MachineLearning/bert-large-portuguese-cased_pytorch_checkpoint"
model = BertForMaskedLM.from_pretrained(model_path)
model.eval()

In [ ]:
tokeniser = BertTokenizer.from_pretrained(model_path)
f = open("D:/MachineLearning/Scooby/scoobyCompendiumText.txt", "r")
text = "dummy. " + f.read()
tokenised_text = tokeniser.tokenize(text)

f = open("D:/MachineLearning/Scooby/scoobyCompendiumText_keywords.txt", "r")
keywords = f.read().split()

In [ ]:
punctuation = [
    ',',
    '.',
    '!',
    '?',
    ':',
    ';',
    '...',
    '\n', 
    '\n\n'
]

In [ ]:
text = replace_tokens(text, tokenised_text, keywords, punctuation, "selection")

In [ ]:
print(text.replace('dummy. ','', 1))

# GERADOR V1.2

In [ ]:
# Tokenizer 
from transformers import T5Tokenizer

# PyTorch (bare model, baremodel + language modeling head)
from transformers import T5Model, T5ForConditionalGeneration

model_name = 'unicamp-dl/ptt5-base-portuguese-vocab'

tokeniser = T5Tokenizer.from_pretrained(model_name)

# PyTorch
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
f = open("D:/MachineLearning/Scooby/scoobyCompendiumText.txt", "r")
text = "dummy. " + f.read()
tokenised_text = tokenizer.tokenize(text)

f = open("D:/MachineLearning/Scooby/scoobyCompendiumText_keywords.txt", "r")
keywords = f.read().split()

text = replace_tokens(text, tokenised_text, keywords, punctuation, "selection")

In [ ]:
print(text.replace('dummy. ','', 1))

# GERADOR V2
With GPT-2

In [ ]:
import gpt_2_simple as gpt2

sess = gpt2.start_tf_sess()
gpt2.finetune(sess, 
              'D:/MachineLearning/Scooby/scoobyCompendiumText.txt', 
              model_name="124M",
              checkpoint_dir="checkpoint-wiki/run1",
              steps=1000)

gpt2.generate(sess)

In [ ]:
single_text = gpt2.generate(sess, return_as_list=True)[0]
print(single_text)

In [ ]:
single = gpt2.generate(sess, 
              length=40,
              include_prefix=False, 
              temperature=0.1, 
              top_k=1, 
              top_p=0.9, 
              run_name='run1', 
              prefix="O gato ou rato, quem chegou aqui primeiro?", 
              return_as_list=True)[0]
print(single)

In [ ]:
import gpt_2_simple as gpt2

sess = gpt2.start_tf_sess()
gpt2.finetune(sess, 
              'D:/MachineLearning/Scooby/scoobyCompendiumText.txt', 
              model_name="124M",
              steps=1000)

In [ ]:
single_text = gpt2.generate(sess, return_as_list=True)[0]
print(single_text)

#  GERADOR V2.1

In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

# GERADOR V2.2

## Build Dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re

In [2]:
df = pd.read_csv('D:/MachineLearning/Scooby/training.txt', sep='\t', names=['summary'])
df

,summary
0,"A redenção não impõe o momento, mas se transfo..."
1,Um forte abraço boa semana pra você e sua famí...
2,"Pensar, agir! Quando invertida a ordem, pode n..."
3,Toda ação terá bons resultados e reconheciment...
4,"Desejo, a você, uma ótima semana. Forte abraço!"
...,...
253,Somos produto de nossas atitudes.
254,Quando as situações na qual fazemos parte apen...
255,Um forte abraço e uma ótima semana.
256,Tudo que se baseia na confiança deve ser trata...


In [3]:
train_test_ratio = 0.9
train_valid_ratio = 7/9
df_full_train, df_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)
df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [4]:
import codecs

In [5]:
def build_dataset(df, dest_path):
    f = codecs.open(dest_path, 'w', "utf-8")
    data = ''
    summaries = df['summary'].tolist()
    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        #bos_token = '<BOS>'
        #eos_token = '<EOS>'
        #data += bos_token + ' ' + summary + ' ' + eos_token + '\n'
        data += summary + '\n'
        
    f.write(data)

In [6]:
build_dataset(df, 'D:/MachineLearning/Scooby/newTrain.txt')

In [7]:
build_dataset(df_train, 'D:/MachineLearning/Scooby/train.txt')
build_dataset(df_valid, 'D:/MachineLearning/Scooby/valid.txt')
build_dataset(df_test, 'D:/MachineLearning/Scooby/test.txt')

In [8]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train":'D:/MachineLearning/Scooby/newTrain.txt', "validation":'D:/MachineLearning/Scooby/valid.txt'})

Using custom data configuration default-579769b0bc351d3c


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to C:\Users\GiovaneMorbi\.cache\huggingface\datasets\text\default-579769b0bc351d3c\0.0.0\e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,text
0,Uma ótima semana.
1,Uma ótima semana. Forte abraço!
2,Que sua semana seja fantástica. Forte abraço!
3,"As águas de um rio sempre correm para o mesmo sentido, sempre para baixo, tentar fazer algo para mudar seria insano."
4,A subjetividade da beleza é o que a torna mais interessante.
5,Forte abraço!
6,Uma ótima semana para você. Forte abraço!
7,Quanto vale um pai?
8,"A lâmpada queimada no fundo do corredor já não importa tanto, afinal pra que luz se o alcance da vista já se limita a poucos passos."
9,"As fotos guardadas, agora parecem fazer mais sentido. Observe que seu amigo ainda tinha cabelos, os seus já estão brancos. Quem é aquele da direita? Ele se foi aos vinte e dois anos de idade, vítima de um trágico acidente."


# Modeling

In [11]:
model_checkpoint = "D:/MachineLearning/Scooby/gpt2-small-portuguese"

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [14]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=1, remove_columns=["text"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
tokenized_datasets["train"][1]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [2635,
  2163,
  12501,
  645,
  3379,
  2778,
  1281,
  4061,
  258,
  450,
  1373,
  14]}

In [16]:
#block_size = tokenizer.model_max_length
block_size = 100

In [17]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [18]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=12,
    num_proc=1,
)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [19]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'Desejo à você,  uma ótima semana. Abração!A hipocrisia e a falsidade vivem nos bastidores das melhores intenções, até que um dia as cortinas caem.Desejo lhe uma ótima semana. Forte abraço!Somos o únicos seres terrenos que pensam no amanhã,  ao mesmo tempo não temos certeza nenhuma desse futuro, e a dúvida persiste.Como pode, o velho elefante prever seu derradeiro destino?Que sua semana seja a melhor possível!Atingir o topo'

In [20]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [21]:
from transformers import Trainer, TrainingArguments

In [22]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

In [23]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [24]:
trainer.train()

***** Running training *****
  Num examples = 25
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: gmorbi (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss
1,No log,4.928629
2,No log,4.788982
3,No log,4.740630


***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=12, training_loss=4.775369962056478, metrics={'train_runtime': 11.1742, 'train_samples_per_second': 6.712, 'train_steps_per_second': 1.074, 'total_flos': 3827520000000.0, 'train_loss': 4.775369962056478, 'epoch': 3.0})

In [25]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 4
  Batch size = 8


Perplexity: 114.51


In [26]:
trainer.save_model("D:/MachineLearning/Scooby/scoobymodel")

Saving model checkpoint to D:/MachineLearning/Scooby/scoobymodel
Configuration saved in D:/MachineLearning/Scooby/scoobymodel\config.json
Model weights saved in D:/MachineLearning/Scooby/scoobymodel\pytorch_model.bin


# Validate

In [27]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

tokenizer = AutoTokenizer.from_pretrained("D:/MachineLearning/Scooby/gpt2-small-portuguese")
model = AutoModelWithLMHead.from_pretrained("D:/MachineLearning/Scooby/scoobymodel")

# Get sequence length max of 1024
tokenizer.model_max_length=1024 

model.eval()  # disable dropout (or leave in train mode to finetune)

loading configuration file D:/MachineLearning/Scooby/gpt2-small-portuguese\config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.10.0.dev0",
  "use_cache": true,
  "vocab_size": 50257
}

Didn't find file D:/MachineLearning/Scooby/gpt2-small-portuguese\tokenizer.j

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [59]:
text = "Cerveja artesanal,"
inputs = tokenizer(text, return_tensors="pt")

# model output using Top-k sampling text generation method
sample_outputs = model.generate(inputs.input_ids,
                                pad_token_id=50256,
                                do_sample=True, 
                                max_length=100, # put the token number you want
                                top_k=40,
                                num_return_sequences=1)

# generated sequence
for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))

>> Generated text 1

Cerveja artesanal, com base na cultura regional, com grande tradição em nossa região". Em sua página do Facebook, a marca, que tem como principal objetivo, desenvolver mais uma experiência de vida, da região e da cidade, o que lhe garante a garantia.

A cultura turística sempre estará à frente dos visitantes e sempre estará presente em qualquer atividade. É tudo dentro da casa, desde que todos tenham uma boa ideia da existência e o que pode oferecer, e que podem oferecer a melhor experiência.


wandb: Network error resolved after 13:34:36.141836, resuming normal operation.
